In [1]:
import gurobipy as gp
from gurobipy import GRB

In [5]:

# Parameters dictionary
data = {
    'days': list(range(1, 8)),           # Days 1 (Mon) to 7 (Sun)
    'demand': 5000,                      # Weekly delivery demand
    'defect_rate': 0.01,                 # Defective rate
    'unit_cost': 3000,                   # Cost per produced unit
    'num_workers': 12,
    'weekly_wage_per_worker': 1500,
    'storage_cost_per_unit_per_day': 10,
    'shipping_cost_per_unit': 5,
    'min_inventory': 3000,
    'max_inventory': 6000,
    'max_daily_production': 1650,
    'delivery_day': [1,2,3,4,5]                    
}

# Initial inventory at start of week (assumed)
data['initial_inventory'] = 3400
delievery_days = data['delivery_day']

In [6]:
def optimize_production(params):
    days = params['days']
    model = gp.Model('WeeklyProduction')

    x = model.addVars(days, name='prod', lb=0, ub=params['max_daily_production'])
    I = model.addVars(days, name='inv', lb=params['min_inventory'], ub=params['max_inventory'])
    S = model.addVars(days, name='ship', lb=0, ub=params['demand'])

    # Inventory dynamics
    for d in days:
        if d == 1:
            prev_inv = params['initial_inventory']
        else:
            prev_inv = I[d-1]
        model.addConstr(
            I[d] == prev_inv + x[d] * (1 - params['defect_rate']) - S[d],
            name=f'inv_balance_{d}'
        )

    # Total shipped over delivery days must meet demand
    model.addConstr(gp.quicksum(S[d] for d in delievery_days) >= params['demand'], name='total_ship')

    # Shipments cannot exceed available inventory on delivery days
    for d in delievery_days:
        if d == 1:
            available = params['initial_inventory'] + x[d] * (1 - params['defect_rate'])
        else:
            available = I[d-1] + x[d] * (1 - params['defect_rate'])
        model.addConstr(S[d] <= available, name=f'ship_capacity_{d}')

    prod_cost = gp.quicksum(x[d] * params['unit_cost'] for d in days)
    wages = params['num_workers'] * params['weekly_wage_per_worker']
    storage = gp.quicksum(I[d] * params['storage_cost_per_unit_per_day'] for d in days)
    ship_cost = gp.quicksum(S[d] * params['shipping_cost_per_unit'] for d in delievery_days)
    model.setObjective(prod_cost + wages + storage + ship_cost, GRB.MINIMIZE)

    model.optimize()

    if model.status == GRB.OPTIMAL:
        prod_plan = {d: x[d].X for d in days}
        total_cost = model.ObjVal
        return prod_plan, total_cost
    else:
        print(f"Optimization ended with status {model.status}: {model.Status}")
        return None, None


In [7]:
plan, cost = optimize_production(data)
if plan is not None and cost is not None:
    print('Optimal Daily Production:')
    for d, qty in plan.items():
        print(f' Day {d}: {qty:.0f} units')
    print(f'Weekly Minimum Cost: {cost:,.2f} CNY')
else:
    print("Optimization did not find a feasible solution.")


Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 13 rows, 21 columns and 46 nonzeros
Model fingerprint: 0x561f5849
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 3e+03]
  Bounds range     [2e+03, 6e+03]
  RHS range        [3e+03, 5e+03]
Presolve removed 3 rows and 6 columns
Presolve time: 0.01s
Presolved: 10 rows, 15 columns, 36 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2800000e+05   6.750000e+02   0.000000e+00      0s
       6    1.4192394e+07   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.419239394e+07
Optimal Daily Production:
 Day 1: 